<a href="https://www.kaggle.com/code/soumyasangamjha/mrpc-final-f?scriptVersionId=156907149" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
import pandas as pd
from datasets import load_dataset
mrpc_dataset = load_dataset('glue', 'mrpc')
train_df = pd.DataFrame({'input_text': mrpc_dataset['train']['sentence1'], 'target_text': mrpc_dataset['train']['sentence2']})
test_df = pd.DataFrame({'input_text': mrpc_dataset['test']['sentence1'], 'target_text': mrpc_dataset['test']['sentence2']})
df = pd.concat([train_df, test_df], ignore_index=True)



Generating train split:   0%|          | 0/3668 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/408 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1725 [00:00<?, ? examples/s]

Dataset glue downloaded and prepared to /root/.cache/huggingface/datasets/glue/mrpc/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [2]:
df.head()

,input_text,target_text
0,"Amrozi accused his brother , whom he called "" ...","Referring to him as only "" the witness "" , Amr..."
1,Yucaipa owned Dominick 's before selling the c...,Yucaipa bought Dominick 's in 1995 for $ 693 m...
2,They had published an advertisement on the Int...,"On June 10 , the ship 's owners had published ..."
3,"Around 0335 GMT , Tab shares were up 19 cents ...","Tab shares jumped 20 cents , or 4.6 % , to set..."
4,"The stock rose $ 2.11 , or about 11 percent , ...",PG & E Corp. shares jumped $ 1.63 or 8 percent...


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5393 entries, 0 to 5392
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   input_text   5393 non-null   object
 1   target_text  5393 non-null   object
dtypes: object(2)
memory usage: 84.4+ KB


In [4]:
from transformers import pipeline
import tensorflow as tf
model = pipeline('text-classification', model='SamLowe/roberta-base-go_emotions', top_k=None)
positive_grad=['amusement','excitement','pride','optimism','gratitude','joy','admiration','love','approval','caring','desire','relief']
neutral_grad=['confusion','curiosity','realization','surprise','neutral']
negative_grad=['anger','disgust','grief','fear','sadness','nervousness','annoyance','disappointment','embarrassment','remorse','disapproval']
total_grad=positive_grad+neutral_grad+negative_grad

config.json:   0%|          | 0.00/1.92k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/380 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

In [5]:
df.head(6)

,input_text,target_text
0,"Amrozi accused his brother , whom he called "" ...","Referring to him as only "" the witness "" , Amr..."
1,Yucaipa owned Dominick 's before selling the c...,Yucaipa bought Dominick 's in 1995 for $ 693 m...
2,They had published an advertisement on the Int...,"On June 10 , the ship 's owners had published ..."
3,"Around 0335 GMT , Tab shares were up 19 cents ...","Tab shares jumped 20 cents , or 4.6 % , to set..."
4,"The stock rose $ 2.11 , or about 11 percent , ...",PG & E Corp. shares jumped $ 1.63 or 8 percent...
5,Revenue in the first quarter of the year dropp...,With the scandal hanging over Stewart 's compa...


In [6]:
def predict_emotion(text):
    model_output = model([text])
    predictions = model_output[0]
    
    max_score_index = max(range(len(predictions)), key=lambda i: predictions[i]["score"])
    return predictions[max_score_index]["label"]


In [7]:
from tqdm import tqdm

for index, row in tqdm(df.iterrows(), total=len(df), desc='Processing rows'):
    input_text = row['input_text']
    target_text = row['target_text']

    input_emotion = predict_emotion(input_text)
    target_emotion = predict_emotion(target_text)

    if (input_emotion == target_emotion or input_emotion == 'neutral' or target_emotion == 'neutral'):
        df.drop(index, inplace=True)
    else:
        if total_grad.index(input_emotion) > total_grad.index(target_emotion):
            df.at[index, 'input_text'], df.at[index, 'target_text'] = df.at[index, 'target_text'], df.at[index, 'input_text']
            input_emotion, target_emotion = target_emotion, input_emotion
        emotion_transition = f"{input_emotion} to {target_emotion}: "
        modified_input_text = emotion_transition + input_text
        df.at[index, 'input_text'] = modified_input_text


Processing rows: 100%|██████████| 5393/5393 [13:10<00:00,  6.82it/s]


In [8]:
df.to_csv('mrpc_3.csv',index=False)

In [9]:
df.head(5)

,input_text,target_text
25,admiration to desire: I wanted to bring the mo...,I wanted to bring the most beautiful people in...
163,"excitement to joy: For the weekend , the top 1...","For the weekend , the top 12 movies grossed $ ..."
258,surprise to disappointment: Revelations of the...,Revelations of the expenditures shocked some e...
269,amusement to caring: A sign outside the Peacht...,A sign outside the Peachtree Restaurant reads ...
642,sadness to disappointment: For the third time ...,"For the third time in the past four years , wi..."


In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 28 entries, 25 to 5113
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   input_text   28 non-null     object
 1   target_text  28 non-null     object
dtypes: object(2)
memory usage: 672.0+ bytes
